This is a document I use to separate out functions of the main script in order to troubleshoot or quickly bypass some steps. I often use this to avoid spamming chat or API requests when I want to test changes in a script without using quota credits or some other resource.

In [4]:
from funcs import *
import requests,time
from pprint import pprint
showbot_channel,token = 'Frogpants','Nhrj6amcz4AqiSP6AVv5YhhQX8OhJ6wO'
channel_id = '@ScottJohnson'
numeric_id = 'UCIEIRz-KpYoEPnrNQuyHwJw'

In [8]:
! ".venv/Scripts/activate.bat"

In [6]:
streamid = find_stream_id('@ScottJohnson')
if streamid:
    print(f'Live stream detected.\nLive stream id: {streamid}')
else:
    print('No live stream found.')

Live stream detected.
Live stream id: fsTfWrRMDAs


In [6]:
streamid = 'fsTfWrRMDAs'

For testing

In [ ]:
authors,messages = parse_chat(read_chat(find_chat_id(streamid)))
for author,message in zip(authors,messages):
    print(f'{author}: {message}')

For titles

In [ ]:
author_index,titles = title_search(read_chat(find_chat_id(streamid)))
# for author,title in zip(authors,titles):
#     print(f'{author}: {title}')

In [18]:
authors_html,titles_html = [html_ify(auth) for auth in author_index],[html_ify(title) for title in titles]
for author_html,title_html in zip(authors_html,titles_html):
    print(f'{author_html}: {title_html}')
    

In [7]:
livechatid = find_chat_id(streamid)

Credentials found.


In [12]:
livechatid = 'Cg0KC3Rhb0lYcU9Udi1jKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILdGFvSVhxT1R2LWM'

In [9]:
author_index,submitted_titles = build_submission_history(showbot_channel)

Titles scraped for Showbot.tv channel: Frogpants 


In [10]:
for author,title in zip(authors,titles):
    print(f'{author}: {title}')

NameError: name 'authors' is not defined

In [10]:
credentials = build_credentials(client_secrets_file,scopes,testing=True)
credentials

Credentials found.
Credentials refreshed.
New credentials saved via default flow.


In [3]:
credentials = Credentials.from_authorized_user_file("../working_submit_token_Talia_do_not_delete.json", scopes)
youtube = build_yt_obj(credentials,api_service_name, api_version,module=googleapiclient.discovery)

In [16]:
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file,scopes)
credentials = flow.run_local_server(port=0)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=347817049340-9r6vmp3vflu05g81jt897f3hankoud92.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58104%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=QKBk0bXll44SuYzoaI2JNYLwZQg7cl&access_type=offline


In [3]:
credentials = build_credentials(client_secrets_file,scopes)

Credentials found.


In [17]:
youtube = build_yt_obj(credentials,api_service_name, api_version,module=googleapiclient.discovery)
youtube

In [15]:
with open('../token.json','w') as token:
    token.write(credentials.to_json())

In [9]:
try:
    send_chat_message('Cg0KC3Rhb0lYcU9Udi1jKicKGFVDSUVJUnotS3BZb0VQbnJOUXV5SHdKdxILdGFvSVhxT1R2LWM','Dicks')
except googleapiclient.errors.HttpError:
    print('No live chat detected, likely because the chat has ended.')

Original Token Used.
No live chat detected, likely because the chat has ended.


In [ ]:
while True:
    authors,titles = title_search(read_chat(livechatid)) #Poll API for all chat messages, regardless of they've been polled already or not. title_search() already filters for '!s' chat trigger and cleans as needed
    #Check extracted titles versus previously submitted ones, submit if new
    for author,title in zip(authors,titles):
        if title not in submitted_titles: #Check extracted titles versus previously submitted ones, submit if new
            author_index.append(author)
            submitted_titles.append(title)
            print(f'{author}: {title}') #Add new title to list and add author to another list to maintain the order. Print for debugging and monitoring purposes.
            author_html,title_html = html_ify(author),html_ify(title) #Prepare link for submission then submit via Requests module
            submission_link = f'http://www.showbot.tv/s/add.php?title={title_html}&user={author_html}&channel={showbot_channel}&key={token}'
            requests.get(submission_link)
            #At this point the title has been submitted. This is one complete function and API call. Confirmation via the .insert() method to send a message is a separate 
            #function incurring higher quota useage.
            credentials = build_credentials(client_secrets_file,scopes) #Builds OAuth2 credentials object. Uses existing files or makes new one if not detected. This links the function to the account that follows the URI.
            youtube = build_yt_obj(googleapiclient.discovery,credentials) #Common step in calling the API. Always have to build an object on which to call methods.
            response = send_chat_message(livechatid,author) #The API response in JSON format. Not technically needed but saved in case.
            print(f'Title successfully submitted and confirmed: {author}') #Generic success message only for the console for debugging and monitoring purposes.
    time.sleep(5)

In [11]:
author_index,submitted_titles = build_submission_history(showbot_channel)

In [ ]:
while True:
    authors,titles = title_search(read_chat(livechatid)) #Poll API for all chat messages, regardless of they've been polled already or not. title_search() already filters for '!s' chat trigger and cleans as needed
    #Check extracted titles versus previously submitted ones, submit if new
    for author,title in zip(authors,titles):
        if title not in submitted_titles: #Check extracted titles versus previously submitted ones, submit if new
            author_index.append(author)
            submitted_titles.append(title)
            print(f'{author}: {title}') #Add new title to list and add author to another list to maintain the order. Print for debugging and monitoring purposes.
            author_html,title_html = html_ify(author),html_ify(title) #Prepare link for submission then submit via Requests module
            submission_link = f'http://www.showbot.tv/s/add.php?title={title_html}&user={author_html}&channel={showbot_channel}&key={token}'
            try:
                requests.get(submission_link)
            except TimeoutError:
                print('Timeout error')
            #At this point the title has been submitted. This is one complete function and API call. Confirmation via the .insert() method to send a message is a separate 
            #function incurring higher quota useage.
            credentials = Credentials.from_authorized_user_file("../working_submit_token_Talia_do_not_delete.json", scopes)
            youtube = build_yt_obj(googleapiclient.discovery,credentials,api_service_name, api_version) #Common step in calling the API. Always have to build an object on which to call methods.
            try:
                response = send_chat_message(livechatid,author) #The API response in JSON format. Not technically needed but saved in case.
                print(f'Title successfully submitted and confirmed: {author}') #Generic success message only for the console for debugging and monitoring purposes.
            except googleapiclient.errors.HttpError: #For some reason the script can get a 403 response code error when attempting to submit a confirmation response. This block keeps the script from halting if that occurs.
                print(f'HTTP Error: [{author}: {title}] failed to submit.')# Trying again in 3 seconds.') #This is an experimental approach to handling the 403 error by attempting to resubmit the response asynchronously.
                # asyncio.sleep(3)
                # try:
                #     response = asyncio.send_async_chat_message(livechatid,author)
                # except googleapiclient.errors.HttpError:
                #     print(f'Submission of [{author}: {title}] failed.')
                # print(traceback.format_exc())
    time.sleep(5) 

In [4]:
credentials = build_credentials(client_secrets_file,scopes) #Builds OAuth2 credentials object. Uses existing files or makes new one if not detected. This links the function to the account that follows the URI.
youtube = build_yt_obj(credentials) #Common step in calling the API. Always have to build an object on which to call methods.

Credentials found.
Credentials rebuilt via error.
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=347817049340-9r6vmp3vflu05g81jt897f3hankoud92.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=7D4cyVk2aBsqFxENfOuuWEeVnmJheF&access_type=offline
New credentials saved via function.


In [13]:
message_raw = ['TMS is a collaborative effort! ']
i = 0
for patreon in patreon_links:
    i += 1
    link = patreon_links[patreon]
    if patreon == 'TMS':
        message = f'Support TMS directly at {link}, '
    elif patreon == 'Frogpants':
        message = f'or support the whole network at: {link}. Support individuals: '
    elif i == len(patreon_links):
        message = f' and {patreon} at {link}.'
    else:
        message = f' {patreon} at {link},'
    message_raw.append(message)

In [12]:
!python -m pip install atproto

  Using cached atproto-0.0.55-py3-none-any.whl.metadata (14 kB)
  Using cached cryptography-43.0.3-cp39-abi3-win_amd64.whl.metadata (5.4 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached libipld-3.0.0-cp312-none-win_amd64.whl.metadata (3.6 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached websockets-13.1-cp312-cp312-win_amd64.whl.metadata (7.0 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.4-cp312-none-win_amd64.whl.metadata (6.7 kB)
Using cached atproto-0.0.55-py3-none-any.whl (327 kB)
Using cached cryptography-43.0.3-cp39-abi3-win_amd64.whl (3.1 MB)
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached libipld-3.0.0-cp312-none-win_amd64.whl (192 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from atproto import Client

client = Client()

In [2]:
import os
os.getcwd()

'c:\\Users\\RyuRa\\OneDrive\\Github\\Frogpants_Showbot'

In [6]:
with open('../cache_pass.txt','r') as f:
    cache_pass = f.readlines()

In [ ]:
response = client.login('TaliZorEl.bsky.social', cache_pass)

NameError: name 'client' is not defined

In [26]:
import pprint

In [31]:
pprint.pprint(response)

("did='did:plc:lwzxfxdwbwyjrgwpuper7lk4' handle='talizorel.bsky.social' "
 'associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, '
 "starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated') "
 "avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:lwzxfxdwbwyjrgwpuper7lk4/bafkreicud3sbrvcjr7s24occmgqscuwou443aaolme5gyr7cuiwbjdbe54@jpeg' "
 "banner='https://cdn.bsky.app/img/banner/plain/did:plc:lwzxfxdwbwyjrgwpuper7lk4/bafkreigavjbsunhvfbdykcjqbwh5hlwczex6gmje3mgw4o35pqdaj4ardi@jpeg' "
 "created_at='2024-11-19T20:48:06.599Z' description='Former Research "
 'Neuroscientist || Current Code Monkey for Pennies || Trans || Pathologic '
 "Gamer || Chronic TTRPGer' display_name='Talia' followers_count=0 "
 "follows_count=8 indexed_at='2024-11-19T21:08:34.098Z' "
 'joined_via_starter_pack=None labels=[] pinned_post=None posts_count=1 '
 'viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, '
 'followed_by=None, following=None, known_follower

In [32]:
response = client.send_post(text="Well if this posts, then it means I've already figured out some of the BlueSky API and now people should be scared.")